In [1]:
import os
import urllib.request, urllib.parse, urllib.error
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as soup
import time
import json
from collections import OrderedDict

In [2]:
import pandas as pd

In [5]:
#함수 작성
def insta_searching(word):  #word라는 매개변수를 받는 insta_searching 이라는 함수 생성
    url = 'https://www.instagram.com/explore/tags/' + word
    return url
 
#열린 크롬으로 개발자 도구 활용하여 첫번째 게시물 태그 확인 (<div class="_9AhH0"></div>)
#첫번째 게시물 찾아 클릭 함수 만들기

import time
 
def select_first(driver):
    first = driver.find_element_by_css_selector('div._9AhH0') #find_element_by_css_selector 함수를 사용해 요소 찾기
    first.click()
    time.sleep(3) #로딩을 위해 3초 대기
 
#본문 내용, 작성 일시, 위치 정보 및 해시태그(#) 추출
 
import re
 
def get_content(driver):
    # 1. 현재 페이지의 HTML 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    
    # 2. 본문 내용 가져오기
    try:                                         #여러 태그중 첫번째([0]) 태그를 선택  
        content = soup.select('div.C4VMK > span')[0].text #첫 게시글 본문 내용이 <div class="C4VMK"> 임을 알 수 있다.
                                #태그명이 div, class명이 C4VMK인 태그 아래에 있는 span 태그를 모두 선택.
    except:
        content = ' ' 
        
    # 3. 본문 내용에서 해시태그 가져오기(정규표현식 활용)
    tags = re.findall(r'#[^\s#,\\]+', content) 
        # content 변수의 본문 내용 중 #으로 시작하며, #뒤에 연속된 문자(공백이나 #, \ 기호가 아닌 경우)를 모두 찾아 tags 변수에 저장
    
    # 4. 작성 일자 가져오기
    try:
        date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10] #앞에서부터 10자리 글자
    except:
        date = ''
 
    # 5. 좋아요 수 가져오기
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1] 
    except:
        like = 0
        
    # 6. 위치 정보 가져오기
    try:
        place = soup.select('div.JF9hh')[0].text
    except:
        place = ''
    
    # 7. 수집한 정보 저장하기
    data = [date, content, like, tags]
    return data    

# content, date, like, place

def move_next(driver):
    right = driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow') 
    right.click()
    time.sleep(3)
    
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

# 1. 크롬으로 인스타그램 - '국내여행지추천' 검색
driver = wd.Chrome(executable_path='./chromedriver')
word = '국내여행'
url = insta_searching(word)
driver.get(url) 
time.sleep(4)

# 2. 로그인 하기
login_section = '//*[@id="react-root"]/section/nav/div[2]/div/div/div[3]/div/span/a[1]/button'
driver.find_element_by_xpath(login_section).click()
time.sleep(3)


elem_login = driver.find_element_by_name("username")
elem_login.clear()
elem_login.send_keys('goeunseong12')
 
elem_login = driver.find_element_by_name('password')
elem_login.clear()
elem_login.send_keys('gp941105**')
 
time.sleep(1)
 
xpath = """//*[@id="loginForm"]/div/div[3]/button/div"""
driver.find_element_by_xpath(xpath).click()
 
time.sleep(4)

xpath1 = """//*[@id="react-root"]/section/main/div/div/div/div/button"""

driver.find_element_by_xpath(xpath1).click()
time.sleep(4)

# 3. 검색페이지 접속하기
driver.get(url)
time.sleep(4)

# 4. 첫번째 게시글 열기
select_first(driver)

# 5. 비어있는 변수(results) 만들기
results = []
 
# 여러 게시물 크롤링하기
target = 1000 #크롤링할 게시물 수
for i in range(target):
    data = get_content(driver) #게시물 정보 가져오기
    results.append(data)
    move_next(driver)
    if i%100 == 0:
        print(i)

    
print(results[:2])


# #data = [content, date, like, place, tags]

import pandas as pd
df = pd.DataFrame(results)
df.to_csv('여행지.csv',index=False, encoding='utf-16')

              
# # df['content']= content
# # df['date'] = date
# # df['like'] = like
# # df['place'] = place
# df['tags'] = tags
# # df['date'] = pd.to_datetime(df['date']) 
# df.to_csv('여행지.csv',index=False)
# return df



0
100
200
300
400
500
600
700
800
900
[['2020-08-09', '우리\U0001f90d👫🏻연기중인 나와 어색한 @no_hooni ㅋㅋ#충주#충주호#캠핑#여행에미치다', 0, ['#충주', '#충주호', '#캠핑', '#여행에미치다']], ['2020-08-09', '‘여기는 경주,황화코스모스’공간이 주는 힘이라는 게 정말 대단하구나.이번 주 촬영답사겸 한 바퀴 돌아보니 경주 포인트 구석구석 카메라 들고 나온 사람들이 엄청 많던... 맑았다가 비가 쏟아졌다가 쌍무지개가 떴다가 정말 다이나믹했던 일요일... 입추도 지났고 이제 어서 가을이 오기를 🌿🏷2020년 #경주스냅 유료촬영 가을시즌 예약중입니다.동화같이 예쁜 경주에서 두 분의 이야기를 담아드립니다.‘야외스냅’과 ‘본식스냅’을 함께 예약하실 경우 10%할인혜택 드립니다. 🔜자세한 상품 구성 안내는 프로필에 기재된 블로그 참고해주세요.🏷 ‘경주웨딩’ 그리고 봄부터 겨울까지 함께 담는 특별한 커스터마이징 #사계절스냅 문의까지 언제나 환영합니다. 🙋🏻\u200d♀️📸🙋🏻\u200d♂️. 경주를 사랑하는...아름다운 경주에 살고 경주를 담습니다.어쩌다

In [7]:
df.columns = ['date', 'content', 'like', 'tag']

In [9]:
df.to_csv('국내여행인스타.csv')